In [71]:
# import requiered libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

In [73]:
!git branch -a

* QC
  feature/extraxt-patient-data
  main
  update/extract-patient-data
  remotes/origin/HEAD -> origin/main
  remotes/origin/feature/extraxt-patient-data
  remotes/origin/main
  remotes/origin/ralf/add-data-overview
  remotes/origin/update/extract-patient-data


In [266]:
# Global vars
parameters = ['bg', 'insulin', 'carbs', 'hr', 'steps', 'cals', 'activity']
time_diffs = [
    '-0:00',
    '-0:05','-0:10','-0:15','-0:20','-0:25','-0:30','-0:35','-0:40','-0:45','-0:50','-0:55','-1:00',
    '-1:05','-1:10','-1:15','-1:20','-1:25','-1:30','-1:35','-1:40','-1:45','-1:50','-1:55','-2:00',
    '-2:05',
    '-2:10',
    '-2:15',
    '-2:20',
    '-2:25',
    '-2:30',
    '-2:35',
    '-2:40',
    '-2:45',
    '-2:50',
    '-2:55',
    '-3:00',
    '-3:05',
    '-3:10',
    '-3:15',
    '-3:20',
    '-3:25',
    '-3:30',
    '-3:35',
    '-3:40',
    '-3:45',
    '-3:50',
    '-3:55',
    '-4:00',
    '-4:05',
    '-4:10',
    '-4:15',
    '-4:20',
    '-4:25',
    '-4:30',
    '-4:35',
    '-4:40',
    '-4:45',
    '-4:50',
    '-4:55',
    '-5:00',
    '-5:05',
    '-5:10',
    '-5:15',
    '-5:20',
    '-5:25',
    '-5:30',
    '-5:35',
    '-5:40','-5:45','-5:50','-5:55'
]

In [75]:
# Read raw train data
train = pd.read_csv( '../data/raw/train.csv' )
train.head()

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [76]:
# Define function to run comparisons
def compare_data( id , metric ):
    # Filter the raw DataFrame
    df_raw = train[ train['p_num'] == id ]
    
    # read the parsed data for the given patient ID
    df_parsed = pd.read_csv( f'../data/interim/{id}_train.csv' )
    
    # generate data frames of unique non-NA values
    val1_df = pd.DataFrame({
        'group':'raw',
        'vals':df_raw[df_raw.columns[df_raw.columns.str.startswith(f'{metric}-')]].melt().value.unique()
    })
    
    val2_df = pd.DataFrame({
        'group':'processed',
        'vals':df_parsed[metric].unique()
    })

    val1_df.sort_values( 'vals', inplace = True )
    val2_df.sort_values( 'vals', inplace = True )
    
    # Calculate differences
    vals_df = pd.concat([val1_df,val2_df],axis = 0 )
    vals_df = vals_df.groupby('group').agg(['mean','std',np.min,np.max])
    vals_df = pd.DataFrame( vals_df )
    checks_df  = pd.DataFrame( vals_df.loc['raw',:]-vals_df.loc['processed',:] )
    checks_df.rename( columns = {0:'diff'} , inplace  =True )
    checks_df['p_id'] = id
    checks_df['diff'] = abs( checks_df['diff'] )
    checks_df.reset_index(inplace=True)
    checks_df.drop('level_0',axis=1,inplace=True)
    checks_df.rename( columns = {'level_1':'metric'} , inplace = True )
    checks_df['feature'] =  metric
    checks_df = checks_df[['p_id','feature','metric','diff']]
    checks_df['diff_check'] = checks_df['diff'] < 1e-6

    # Return final DataFrames
    return checks_df , val1_df , val2_df , df_raw , df_parsed
    

In [79]:
# List of ids to check
ids = ['p01','p02','p03','p04','p05','p06','p10','p11','p12']
metrics = ['bg','insulin','carbs','hr','steps','cals']

# Instantiate an empty list to store results
res_l = list()

# Run a For loop
for id in ids:
    for metric in metrics:
        # print(id,metric) 
        checks_df , _ , _  , _ , _ = compare_data( id , metric )
        res_l.append( checks_df )

In [80]:
# Join the individual DataFrames
res_df = pd.concat( res_l ).drop('diff_check',axis=1)
pivot_df = pd.pivot( res_df , index = ['p_id','feature'] , columns = 'metric' , values ='diff' )
pivot_df.reset_index( inplace = True )
pivot_df = pivot_df[['p_id','feature','mean','std','min','max']]
pivot_df

metric,p_id,feature,mean,std,min,max
0,p01,bg,0.000000,0.000000,0.0,0.0
1,p01,cals,0.000000,0.000000,0.0,0.0
2,p01,carbs,0.000000,0.000000,0.0,0.0
3,p01,hr,0.000000,0.000000,0.0,0.0
4,p01,insulin,0.000000,0.000000,0.0,0.0
5,p01,steps,0.000000,0.000000,0.0,0.0
6,p02,bg,0.000000,0.000000,0.0,0.0
7,p02,cals,0.000000,0.000000,0.0,0.0
8,p02,carbs,0.000000,0.000000,0.0,0.0
9,p02,hr,0.000000,0.000000,0.0,0.0


In [85]:
id = 'p11'
metric = 'cals'

In [87]:
df_raw = train[ train['p_num'] == id ]
df_parsed = pd.read_csv( f'../data/interim/{id}_train.csv' )

In [99]:
print( (df_raw==15.36).any().any() )
print( (df_parsed==15.36).any().any() )

True
False


In [136]:
src_folder = '../data/raw'
patient_num = 'p11'
start_date = datetime(2020, 1, 1)

In [247]:
patient_data , dfs1 , dfs2 = extract_patient_data( train , 'p11', datetime(2020, 1, 1) )

Check 1: True
Check 1b: True
Check 2 debug: False
Check 2: False
Check 3: False
Check 4: False


In [251]:
display( dfs1.head() )
display( dfs2.head() )
print

,p_num,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-01 06:05:00,p11,11.6,11.5,11.7,11.5,11.6,10.3,11.1,11.3,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7
2020-01-01 06:10:00,p11,11.5,11.7,11.5,11.6,10.3,11.1,11.3,11.4,11.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4
2020-01-01 06:15:00,p11,11.7,11.5,11.6,10.3,11.1,11.3,11.4,11.3,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:20:00,p11,11.5,11.6,10.3,11.1,11.3,11.4,11.3,11.4,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:25:00,p11,11.6,10.3,11.1,11.3,11.4,11.3,11.4,11.4,11.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.8


,p_num,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00,bg+1:00
datetime,,,,,,,,,
2020-01-01 06:05:00,p11,9.3,NaN,NaN,NaN,NaN,NaN,NaN,8.7
2020-01-01 06:10:00,p11,9.2,NaN,NaN,NaN,NaN,NaN,NaN,8.4
2020-01-01 06:15:00,p11,9.1,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:20:00,p11,9.1,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:25:00,p11,9.2,NaN,NaN,NaN,NaN,NaN,NaN,7.8


In [257]:
ix = np.where( (dfs1==15.36).any() )[0]
dfs1.columns[ ix ]

Index(['cals-5:55', 'cals-5:50', 'cals-5:45', 'cals-5:40', 'cals-5:35',
       'cals-5:30', 'cals-5:25', 'cals-5:20', 'cals-5:15', 'cals-5:10',
       'cals-5:05', 'cals-5:00', 'cals-4:55', 'cals-4:50', 'cals-4:45',
       'cals-4:40', 'cals-4:35', 'cals-4:30', 'cals-4:25', 'cals-4:20',
       'cals-4:15', 'cals-4:10', 'cals-4:05', 'cals-4:00', 'cals-3:55',
       'cals-3:50', 'cals-3:45', 'cals-3:40', 'cals-3:35', 'cals-3:30',
       'cals-3:25', 'cals-3:20', 'cals-3:15', 'cals-3:10', 'cals-3:05',
       'cals-3:00', 'cals-2:55', 'cals-2:50', 'cals-2:45', 'cals-2:40',
       'cals-2:35', 'cals-2:30', 'cals-2:25', 'cals-2:20', 'cals-2:15',
       'cals-2:10', 'cals-2:05', 'cals-2:00', 'cals-1:55', 'cals-1:50',
       'cals-1:45', 'cals-1:40', 'cals-1:35'],
      dtype='object')

In [287]:
ix_rows = np.where( (dfs1==15.36).any( axis = 1 ) )[0]
dfs2.index[ix_rows]

DatetimeIndex(['2020-01-16 10:10:00', '2020-01-16 10:15:00',
               '2020-01-16 10:20:00', '2020-01-16 10:25:00',
               '2020-01-16 10:30:00', '2020-01-16 10:35:00',
               '2020-01-16 10:40:00', '2020-01-16 10:45:00',
               '2020-01-16 10:50:00', '2020-01-16 10:55:00',
               '2020-01-16 11:00:00', '2020-01-16 11:05:00',
               '2020-01-16 11:10:00', '2020-01-16 11:15:00',
               '2020-01-16 11:20:00', '2020-01-16 11:25:00',
               '2020-01-16 11:30:00', '2020-01-16 11:35:00',
               '2020-01-16 11:40:00', '2020-01-16 11:45:00',
               '2020-01-16 11:50:00', '2020-01-16 11:55:00',
               '2020-01-16 12:00:00', '2020-01-16 12:05:00',
               '2020-01-16 12:10:00', '2020-01-16 12:15:00',
               '2020-01-16 12:20:00', '2020-01-16 12:25:00',
               '2020-01-16 12:30:00', '2020-01-16 12:35:00',
               '2020-01-16 12:40:00', '2020-01-16 12:45:00',
               '2020-01-

NOTE: now we now that the value of 15.36 is exclusively located in the **cals** columns. But oddly , not in **cals-0:00** ???

In [262]:
(dfs2=='15.36').any() 

p_num            False
bg-0:00          False
insulin-0:00     False
carbs-0:00       False
hr-0:00          False
steps-0:00       False
cals-0:00        False
activity-0:00    False
bg+1:00          False
dtype: bool

In [278]:
print( list( dfs1.columns ) )

['p_num', 'bg-5:55', 'bg-5:50', 'bg-5:45', 'bg-5:40', 'bg-5:35', 'bg-5:30', 'bg-5:25', 'bg-5:20', 'bg-5:15', 'bg-5:10', 'bg-5:05', 'bg-5:00', 'bg-4:55', 'bg-4:50', 'bg-4:45', 'bg-4:40', 'bg-4:35', 'bg-4:30', 'bg-4:25', 'bg-4:20', 'bg-4:15', 'bg-4:10', 'bg-4:05', 'bg-4:00', 'bg-3:55', 'bg-3:50', 'bg-3:45', 'bg-3:40', 'bg-3:35', 'bg-3:30', 'bg-3:25', 'bg-3:20', 'bg-3:15', 'bg-3:10', 'bg-3:05', 'bg-3:00', 'bg-2:55', 'bg-2:50', 'bg-2:45', 'bg-2:40', 'bg-2:35', 'bg-2:30', 'bg-2:25', 'bg-2:20', 'bg-2:15', 'bg-2:10', 'bg-2:05', 'bg-2:00', 'bg-1:55', 'bg-1:50', 'bg-1:45', 'bg-1:40', 'bg-1:35', 'bg-1:30', 'bg-1:25', 'bg-1:20', 'bg-1:15', 'bg-1:10', 'bg-1:05', 'bg-1:00', 'bg-0:55', 'bg-0:50', 'bg-0:45', 'bg-0:40', 'bg-0:35', 'bg-0:30', 'bg-0:25', 'bg-0:20', 'bg-0:15', 'bg-0:10', 'bg-0:05', 'bg-0:00', 'insulin-5:55', 'insulin-5:50', 'insulin-5:45', 'insulin-5:40', 'insulin-5:35', 'insulin-5:30', 'insulin-5:25', 'insulin-5:20', 'insulin-5:15', 'insulin-5:10', 'insulin-5:05', 'insulin-5:00', 'insul

In [272]:
debug_df= df_patient[['p_num'] + [f"{parameter}{time_diffs[0]}" for parameter in parameters] + ['bg+1:00']].copy()
debug_df

,p_num,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00,bg+1:00
datetime,,,,,,,,,
2020-01-01 06:05:00,p11,9.3,NaN,NaN,NaN,NaN,NaN,NaN,8.7
2020-01-01 06:10:00,p11,9.2,NaN,NaN,NaN,NaN,NaN,NaN,8.4
2020-01-01 06:15:00,p11,9.1,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:20:00,p11,9.1,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:25:00,p11,9.2,NaN,NaN,NaN,NaN,NaN,NaN,7.8
...,...,...,...,...,...,...,...,...,...
2020-03-29 23:35:00,p11,4.6,0.0,NaN,69.0,0.0,6.0,NaN,4.7
2020-03-29 23:40:00,p11,4.7,0.0,NaN,71.8,NaN,6.0,NaN,4.7
2020-03-29 23:45:00,p11,4.6,0.0,NaN,67.5,NaN,6.0,NaN,4.4


In [245]:


def extract_patient_data(df: pd.DataFrame, patient_num: str, start_date: datetime) -> pd.DataFrame | None:
    if patient_num not in df['p_num'].unique():
        return None

    df_patient = df[df['p_num'] == patient_num]

    # convert the time column to datetime
    df_patient.loc[:, "time"] = pd.to_datetime(df_patient["time"], format="%H:%M:%S").dt.time

    current_date = start_date
    assigned_dates = []
    last_time = None

    for i, row in df_patient.iterrows():
        if last_time is None:
            last_time = row["time"]

        if row["time"] < last_time:
            current_date = current_date + timedelta(days=1)

        assigned_dates.append(current_date)
        last_time = row["time"]

    df_patient = df_patient.copy()
    df_patient.loc[:, "date"] = assigned_dates
    df_patient.loc[:, "datetime"] = df_patient.apply(lambda row: datetime.combine(row['date'], row['time']), axis=1)
    df_patient.set_index("datetime", inplace=True)
    df_patient = df_patient.drop(columns=["date", "time", "id"])

    # get resolution of the data
    initial_resolution_in_seconds = (df_patient.index[1] - df_patient.index[0]).seconds
    initial_resolution_in_minutes = initial_resolution_in_seconds / 60
    initial_resolution = f"{int(initial_resolution_in_minutes)}min"

    # change the frequency to 5 minutes
    full_date_range = pd.date_range(start=df_patient.index.min(), end=df_patient.index.max(), freq='5min')
    df_patient = df_patient.reindex(full_date_range)
    df_patient.index.name = "datetime"

    print(f'Check 1: {(df_patient==15.36).any().any()}')

    dfs1 = df_patient.copy()

    
    # organize the columns
    parameters = ['bg', 'insulin', 'carbs', 'hr', 'steps', 'cals', 'activity']
    time_diffs = [
        '-0:00',
        '-0:05',
        '-0:10',
        '-0:15',
        '-0:20',
        '-0:25',
        '-0:30',
        '-0:35',
        '-0:40',
        '-0:45',
        '-0:50',
        '-0:55',
        '-1:00',
        '-1:05',
        '-1:10',
        '-1:15',
        '-1:20',
        '-1:25',
        '-1:30',
        '-1:35',
        '-1:40',
        '-1:45',
        '-1:50',
        '-1:55',
        '-2:00',
        '-2:05',
        '-2:10',
        '-2:15',
        '-2:20',
        '-2:25',
        '-2:30',
        '-2:35',
        '-2:40',
        '-2:45',
        '-2:50',
        '-2:55',
        '-3:00',
        '-3:05',
        '-3:10',
        '-3:15',
        '-3:20',
        '-3:25',
        '-3:30',
        '-3:35',
        '-3:40',
        '-3:45',
        '-3:50',
        '-3:55',
        '-4:00',
        '-4:05',
        '-4:10',
        '-4:15',
        '-4:20',
        '-4:25',
        '-4:30',
        '-4:35',
        '-4:40',
        '-4:45',
        '-4:50',
        '-4:55',
        '-5:00',
        '-5:05',
        '-5:10',
        '-5:15',
        '-5:20',
        '-5:25',
        '-5:30',
        '-5:35',
        '-5:40',
        '-5:45',
        '-5:50',
        '-5:55'
    ]
    print(f'Check 1b: {(df_patient==15.36).any().any()}')

    df_patient_combined_values = df_patient[['p_num'] + [f"{parameter}{time_diffs[0]}" for parameter in parameters] + ['bg+1:00']].copy()
    print(f'Check 2 debug: {(df_patient_combined_values==15.36).any().any()}')

    dfs2 = df_patient_combined_values.copy()
    
    df_patient_combined_values = df_patient_combined_values.rename(columns={f"{parameter}{time_diffs[0]}": f"{parameter}" for parameter in parameters})
    df_patient_combined_values = df_patient_combined_values.reindex(
        pd.date_range(start=df_patient.index.min() + parse_time_diff(time_diffs[-1]), end=df_patient.index.max(), freq='5min')
    )

    print(f'Check 2: {(df_patient_combined_values==15.36).any().any()}')

    for parameter in parameters:
        for time_diff_id, time_diff_str in enumerate(time_diffs):
            if time_diff_str == '-0:00':
                continue

            if not df_patient.columns.str.contains(f"{parameter}{time_diff_str}").any():
                continue

            time_diff = parse_time_diff(time_diff_str)
            values = df_patient[f"{parameter}{time_diff_str}"].copy()
            values.index = values.index + time_diff
            df_patient_combined_values[parameter] = df_patient_combined_values[parameter].combine_first(values)

    print(f'Check 3: {(df_patient_combined_values==15.36).any().any()}')

    # order the columns
    column_order = ['p_num'] + parameters + ['bg+1:00']
    df_patient_combined_values = df_patient_combined_values[column_order]

    # set patient number and initial resolution
    df_patient_combined_values['p_num'] = patient_num
    df_patient_combined_values['initial_resolution'] = initial_resolution

    print(f'Check 4: {(df_patient_combined_values==15.36).any().any()}')

    return df_patient_combined_values , dfs1 , dfs2



In [138]:
df_patient = df[df['p_num'] == patient_num]

# convert the time column to datetime
df_patient.loc[:, "time"] = pd.to_datetime(df_patient["time"], format="%H:%M:%S").dt.time

current_date = start_date
assigned_dates = []
last_time = None

for i, row in df_patient.iterrows():
    if last_time is None:
        last_time = row["time"]

    if row["time"] < last_time:
        current_date = current_date + timedelta(days=1)

    assigned_dates.append(current_date)
    last_time = row["time"]

df_patient = df_patient.copy()
df_patient.loc[:, "date"] = assigned_dates
df_patient.loc[:, "datetime"] = df_patient.apply(lambda row: datetime.combine(row['date'], row['time']), axis=1)
df_patient.set_index("datetime", inplace=True)
df_patient = df_patient.drop(columns=["date", "time", "id"])

# get resolution of the data
initial_resolution_in_seconds = (df_patient.index[1] - df_patient.index[0]).seconds
initial_resolution_in_minutes = initial_resolution_in_seconds / 60
initial_resolution = f"{int(initial_resolution_in_minutes)}min"

# change the frequency to 5 minutes
full_date_range = pd.date_range(start=df_patient.index.min(), end=df_patient.index.max(), freq='5min')
df_patient = df_patient.reindex(full_date_range)
df_patient.index.name = "datetime"

In [140]:
df_patient

,p_num,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-01 06:05:00,p11,11.6,11.5,11.7,11.5,11.6,10.3,11.1,11.3,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7
2020-01-01 06:10:00,p11,11.5,11.7,11.5,11.6,10.3,11.1,11.3,11.4,11.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4
2020-01-01 06:15:00,p11,11.7,11.5,11.6,10.3,11.1,11.3,11.4,11.3,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:20:00,p11,11.5,11.6,10.3,11.1,11.3,11.4,11.3,11.4,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:25:00,p11,11.6,10.3,11.1,11.3,11.4,11.3,11.4,11.4,11.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-29 23:35:00,p11,7.1,7.2,7.3,7.3,7.1,7.2,7.2,7.1,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.7
2020-03-29 23:40:00,p11,7.2,7.3,7.3,7.1,7.2,7.2,7.1,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.7
2020-03-29 23:45:00,p11,7.3,7.3,7.1,7.2,7.2,7.1,7.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4


In [146]:
(df_patient==15.36).any().any()

np.True_

In [152]:
# organize the columns
parameters = ['bg', 'insulin', 'carbs', 'hr', 'steps', 'cals', 'activity']
time_diffs = [
    '-0:00',
    '-0:05',
    '-0:10',
    '-0:15',
    '-0:20',
    '-0:25',
    '-0:30',
    '-0:35',
    '-0:40',
    '-0:45',
    '-0:50',
    '-0:55',
    '-1:00',
    '-1:05',
    '-1:10',
    '-1:15',
    '-1:20',
    '-1:25',
    '-1:30',
    '-1:35',
    '-1:40',
    '-1:45',
    '-1:50',
    '-1:55',
    '-2:00',
    '-2:05',
    '-2:10',
    '-2:15',
    '-2:20',
    '-2:25',
    '-2:30',
    '-2:35',
    '-2:40',
    '-2:45',
    '-2:50',
    '-2:55',
    '-3:00',
    '-3:05',
    '-3:10',
    '-3:15',
    '-3:20',
    '-3:25',
    '-3:30',
    '-3:35',
    '-3:40',
    '-3:45',
    '-3:50',
    '-3:55',
    '-4:00',
    '-4:05',
    '-4:10',
    '-4:15',
    '-4:20',
    '-4:25',
    '-4:30',
    '-4:35',
    '-4:40',
    '-4:45',
    '-4:50',
    '-4:55',
    '-5:00',
    '-5:05',
    '-5:10',
    '-5:15',
    '-5:20',
    '-5:25',
    '-5:30',
    '-5:35',
    '-5:40',
    '-5:45',
    '-5:50',
    '-5:55'
]

df_patient_combined_values = df_patient[['p_num'] + [f"{parameter}{time_diffs[0]}" for parameter in parameters] + ['bg+1:00']].copy()
df_patient_combined_values = df_patient_combined_values.rename(columns={f"{parameter}{time_diffs[0]}": f"{parameter}" for parameter in parameters})
df_patient_combined_values = df_patient_combined_values.reindex(
    pd.date_range(start=df_patient.index.min() + parse_time_diff(time_diffs[-1]), end=df_patient.index.max(), freq='5min')
)

In [161]:
df_patient_combined_values = df_patient[['p_num'] + [f"{parameter}{time_diffs[0]}" for parameter in parameters] + ['bg+1:00']].copy()
print(
    (df_patient==15.36).any().any(),
    (df_patient_combined_values==15.36).any().any() 
)

True False


In [165]:
df_patient[['p_num']].value_counts()

p_num
p11      24555
Name: count, dtype: int64

In [167]:
[f"{parameter}{time_diffs[0]}" for parameter in parameters]

['bg-0:00',
 'insulin-0:00',
 'carbs-0:00',
 'hr-0:00',
 'steps-0:00',
 'cals-0:00',
 'activity-0:00']

In [169]:
df_patient_combined_values

,p_num,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00,bg+1:00
datetime,,,,,,,,,
2020-01-01 06:05:00,p11,9.3,NaN,NaN,NaN,NaN,NaN,NaN,8.7
2020-01-01 06:10:00,p11,9.2,NaN,NaN,NaN,NaN,NaN,NaN,8.4
2020-01-01 06:15:00,p11,9.1,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:20:00,p11,9.1,NaN,NaN,NaN,NaN,NaN,NaN,8.1
2020-01-01 06:25:00,p11,9.2,NaN,NaN,NaN,NaN,NaN,NaN,7.8
...,...,...,...,...,...,...,...,...,...
2020-03-29 23:35:00,p11,4.6,0.0,NaN,69.0,0.0,6.0,NaN,4.7
2020-03-29 23:40:00,p11,4.7,0.0,NaN,71.8,NaN,6.0,NaN,4.7
2020-03-29 23:45:00,p11,4.6,0.0,NaN,67.5,NaN,6.0,NaN,4.4


In [177]:
parameters

['bg', 'insulin', 'carbs', 'hr', 'steps', 'cals', 'activity']

In [192]:
parse_time_diff(time_diff_str)
values = df_patient[f"{parameter}{time_diff_str}"].copy()
values.index = values.index + time_diff
values
parameter

'bg'

In [181]:
for parameter in parameters:
    for time_diff_id, time_diff_str in enumerate(time_diffs):
        print( parameter , time_diff_str )
        if time_diff_str == '-0:00':
            continue

        if not df_patient.columns.str.contains(f"{parameter}{time_diff_str}").any():
            continue

        time_diff = parse_time_diff(time_diff_str)
        values = df_patient[f"{parameter}{time_diff_str}"].copy()
        values.index = values.index + time_diff
        df_patient_combined_values[parameter] = df_patient_combined_values[parameter].combine_first(values)

print(df_patient_combined_values.columns)

bg -0:00
bg -0:05


KeyError: 'bg'

In [175]:


# order the columns
column_order = ['p_num'] + parameters + ['bg+1:00']
df_patient_combined_values = df_patient_combined_values[column_order]

# set patient number and initial resolution
df_patient_combined_values['p_num'] = patient_num
df_patient_combined_values['initial_resolution'] = initial_resolution

KeyError: 'bg'

In [39]:
# Check specifically on p12 insulin
checks_df , val1_df , val2_df , df_raw , df_parsed = compare_data( 'p11' , 'cals' )
print(val1_df.vals.mean())
print(val2_df.vals.mean())
print( val1_df.shape )
print( val2_df.shape )

20.039764468371466
20.042915824915823
(1487, 2)
(1486, 2)


In [33]:
set(val1_df['vals']) - set(val2_df['vals'])

{15.36, nan}

In [69]:
dfs = df_raw[df_raw.columns[df_raw.columns.str.startswith(f'cals-')]] 
row = np.where( (dfs == 15.36).sum( axis  = 1 ) >0 )[0]
col = np.where( (dfs == 15.36).sum( axis  = 0 ) >0 )[0]
dfs.iloc[row,col]

,cals-5:55,cals-5:50,cals-5:45,cals-5:40,cals-5:35,cals-5:30,cals-5:25,cals-5:20,cals-5:15,cals-5:10,...,cals-2:20,cals-2:15,cals-2:10,cals-2:05,cals-2:00,cals-1:55,cals-1:50,cals-1:45,cals-1:40,cals-1:35
131477,5.95,6.07,5.95,5.95,6.07,5.95,5.95,5.95,5.95,5.95,...,5.95,5.95,6.07,6.07,6.07,8.69,5.95,6.43,6.19,15.36
131478,6.07,5.95,5.95,6.07,5.95,5.95,5.95,5.95,5.95,5.95,...,5.95,6.07,6.07,6.07,8.69,5.95,6.43,6.19,15.36,12.61
131479,5.95,5.95,6.07,5.95,5.95,5.95,5.95,5.95,5.95,6.07,...,6.07,6.07,6.07,8.69,5.95,6.43,6.19,15.36,12.61,7.27
131480,5.95,6.07,5.95,5.95,5.95,5.95,5.95,5.95,6.07,6.07,...,6.07,6.07,8.69,5.95,6.43,6.19,15.36,12.61,7.27,15.47
131481,6.07,5.95,5.95,5.95,5.95,5.95,5.95,6.07,6.07,5.95,...,6.07,8.69,5.95,6.43,6.19,15.36,12.61,7.27,15.47,15.00
131482,5.95,5.95,5.95,5.95,5.95,5.95,6.07,6.07,5.95,6.07,...,8.69,5.95,6.43,6.19,15.36,12.61,7.27,15.47,15.00,11.19
131483,5.95,5.95,5.95,5.95,5.95,6.07,6.07,5.95,6.07,5.95,...,5.95,6.43,6.19,15.36,12.61,7.27,15.47,15.00,11.19,19.39
131484,5.95,5.95,5.95,5.95,6.07,6.07,5.95,6.07,5.95,5.95,...,6.43,6.19,15.36,12.61,7.27,15.47,15.00,11.19,19.39,19.88
131485,5.95,5.95,5.95,6.07,6.07,5.95,6.07,5.95,5.95,5.95,...,6.19,15.36,12.61,7.27,15.47,15.00,11.19,19.39,19.88,6.91
131486,5.95,5.95,6.07,6.07,5.95,6.07,5.95,5.95,5.95,5.95,...,15.36,12.61,7.27,15.47,15.00,11.19,19.39,19.88,6.91,6.43
